In [98]:
import sys
sys.path.append("../")

In [99]:
import numpy as np
from fractions import Fraction
from delay.sequencefinder import find
import delay.calculator as calc
from delay.strategy import fA_HardInterval, fB_HardInterval
from delay.simulation import Simulation, FunctionType
from delay.value import calculateValue
from delay.strategy import *

In [100]:
A1 = np.array([
    [1,   0,   0,   0,   0,   0,   0],
    [1,  -1,  0,   0,   0,   0,   0],
    [0, 1/2, -1/2, 1/4, 1/2,   0,   0],
    [0, 1/4, 1/4,  -1/2,   0,   0,   0],
    [0,   0,   0,   0,  -1, 1/2,   1],
    [0, 1/4, 1/4,   0,   0,  -1,   0],
    [0,   0,   0,   0,   0, 1/2,  -1]
])
A2 = np.array([
    [ -1,   0, 1/2,   0,   0,   0,   0,   0],
    [1/2,  -1/2, 1/4,   0,   0,   0,   0,   0],
    [  0,   0,   1,   0,   0,   0,   0,   0],
    [1/2, 1/2, 1/4,  -1,   0,   0,   0,   0],
    [  0,   0,   0, 1/4,  -1/2,   0,   0,   0],
    [  0,   0,   0, 1/4,   0,  -1,   0,   0],
    [  0,   0,   0,   0,   0,   0,  -1,   1],
    [  0,   0,   0, 1/4,   0,   0,   0,  -1]
])

In [101]:
B1 = np.array([1, 0, 0, 0, 0, 0, 0])
B2 = np.array([0, 0, 2, 0, 0, 0, 0, 0])

In [102]:
s1 = np.linalg.solve(A1, B1)
s2 = np.linalg.solve(A2, B2)

In [103]:
s1

array([1. , 1. , 3. , 2. , 1. , 1. , 0.5])

In [104]:
s2

array([1. , 2. , 2. , 2. , 1. , 0.5, 0.5, 0.5])

In [105]:
print([[str(Fraction(p).limit_denominator(120)) for p in s] for s in [s1, s2]])

[['1', '1', '3', '2', '1', '1', '1/2'], ['1', '2', '2', '2', '1', '1/2', '1/2', '1/2']]


In [106]:
s1 *= 104
s2 *= 104
print(s1, s2)

[104. 104. 312. 208. 104. 104.  52.] [104. 208. 208. 208. 104.  52.  52.  52.]


In [107]:
stationary = [s1[0], sum(s2[0:3]), sum(s1[1:5]), sum(s2[3:7]), sum(s1[5:7]), s2[7]]
stationary

[104.0, 520.0, 728.0, 416.0, 155.99999999999997, 52.0]

In [108]:
-2 * stationary[0] - stationary[1] + stationary[3] + 2 * stationary[4] + 3 * stationary[5]

155.99999999999994

In [109]:
sum(stationary)

1976.0

In [110]:
simulation = Simulation(FunctionType.HARD_INTERVAL, (2, 0), calculateValue, calc.linear(N=10, w=1))
simulation.setN(10)
simulation.setConv(0)
simulation.setDuration(100000) # 10^6
simulation.setD(0)

In [111]:
simulation.run()

In [112]:
simStat = [find(simulation.x, [y]) for y in [-2, -1, 0, 1, 2, 3]]
simStat

[5230, 26289, 36918, 21078, 7853, 2633]

In [113]:
np.array(simStat) / sum(simStat)

array([0.05229948, 0.26288737, 0.36917631, 0.21077789, 0.07852921,
       0.02632974])

In [114]:
np.array(simStat) / 50.46

array([103.64645264, 520.98692033, 731.62901308, 417.71700357,
       155.62822037,  52.17994451])

In [115]:
find(simulation.x, [-2, -1, 0]), find(simulation.x, [0, -1, -2])

(5230, 5230)

In [116]:
find(simulation.x, [0, -1, 0]), find(simulation.x, [-2, -1, 0])/2 + find(simulation.x, [0, -1, 0])/2 + find(simulation.x, [0, 1, 0])/4 + find(simulation.x, [2, 1, 0])/2

(15829, 15798.75)

In [117]:
find(simulation.x, [0, 1, 0]), find(simulation.x, [-2, -1, 0])/4 + find(simulation.x, [0, -1, 0])/4 + find(simulation.x, [0, 1, 0])/2

(10637, 10583.25)

In [118]:
find(simulation.x, [2, 1, 0]), find(simulation.x, [0, 1, 2])/2 + find(simulation.x, [2, 3, 2])

(5220, 5243.0)

In [119]:
find(simulation.x, [2, 3, 2]), find(simulation.x, [2, 1, 0])/2

(2633, 2610.0)

In [120]:
find(simulation.x, [-1, -2, -1]), find(simulation.x, [1, 0, -1])/2 

(5230, 5232.5)

In [121]:
find(simulation.x, [-1, 0, -1]), find(simulation.x, [-1, 0, -1])/2 + find(simulation.x, [-1, -2, -1])/2 + find(simulation.x, [1, 0, -1])/4

(10594, 10528.25)

In [122]:
find(simulation.x, [1, 0, -1]), find(simulation.x, [-1, 0, 1])/4 + find(simulation.x, [1, 0, 1])/2 + find(simulation.x, [1, 2, 1]) + find(simulation.x, [3, 2, 1])

(10465, 10532.25)

In [123]:
find(simulation.x, [-1, 0, 1]), find(simulation.x, [-1, -2, -1])/2 + find(simulation.x, [-1, 0, -1])/2 + find(simulation.x, [1, 0, -1])/4

(10465, 10528.25)

In [124]:
find(simulation.x, [1, 0, 1]), find(simulation.x, [-1, 0, 1])/4 + find(simulation.x, [1, 0, 1])/2

(5392, 5312.25)

In [127]:
find(simulation.x, [1, 2, 1]), find(simulation.x, [-1, 0, 1])/4

(2587, 2616.25)

In [125]:
fA = fA_HardInterval(N=10, d=0)
fB = fB_HardInterval(N=10, d=0)

In [126]:
lin = calc.linear(N=10, w=1)